# CV Assignment# 2
## 20i-1754_Ahmad Masood
## 20i-1843_Faizan Aslam
## 19i-1742 Adnan Yousuf

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import LabelEncoder
from torchvision import transforms as T
from PIL import Image

In [1]:
dat_directory = "E:\FramesDatasetEven"  # dataset directory
BatchSize = 32
NumofClasses = len(os.listdir(dat_directory))
InpSze = 224  # model's input size
epochs_count = 10
NumOfFrames = 15  # Number of frames per video
length_sequence = 5  # Number of consecutive frames for each input sequence

# Data Preprocessing and Augmentation
datatransform = {
    'train': T.Compose([
        T.Resize((InpSze, InpSze)),T.RandomHorizontalFlip(),T.ToTensor(),
        T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': T.Compose([
        T.Resize((InpSze, InpSze)),T.ToTensor(),T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

class CustomVideoDataset(Dataset):
    def __init__(self, dat_directory, transform=None):
        self.dat_directory = dat_directory
        self.transform = transform

        self.ClassFolders = os.listdir(dat_directory)
        self.data = []
        self.targets = []

        for i, ClassFolder in enumerate(self.ClassFolders):
            print("\nClassFolder: ", ClassFolder, "\n")
            VideoFolderss = os.listdir(os.path.join(dat_directory, ClassFolder))
            counter=0
            for video_folder in VideoFolderss:
                
                video_path = os.path.join(dat_directory, ClassFolder, video_folder)
                frame_files = sorted(os.listdir(video_path))  #Sort to ensure frame order
                
                for i in range(0, len(frame_files), length_sequence): # Collect frames for each sequence
                    frame_sequence = frame_files[i:i+length_sequence]
                    frame_paths = [os.path.join(video_path, frame) for frame in frame_sequence]
                    self.data.append(frame_paths)
                    self.targets.append(ClassFolder)  #Changed to append 'i' as the target
                    print("Targets: ", ClassFolder, counter)
                    counter+=1

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        frame_sequence = [Image.open(frame_path) for frame_path in self.data[idx]]
        if self.transform:
            frame_sequence = [self.transform(frame) for frame in frame_sequence]
        target = self.targets[idx]  # Get the target
        return frame_sequence, target  # Return as a tuple

# Create datasets and Data_Loader
ImageDataset = {x: CustomVideoDataset(dat_directory, transform=datatransform[x]) for x in ['train', 'val']}
Data_Loader = {x: DataLoader(ImageDataset[x], BatchSize=BatchSize, shuffle=True) for x in ['train', 'val']}


C:\Users\Adnan Yousaf\anaconda3\envs\GPU_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



Class_folder:  abseiling 

Targets:  abseiling 0
Targets:  abseiling 1
Targets:  abseiling 2
Targets:  abseiling 3
Targets:  abseiling 4
Targets:  abseiling 5
Targets:  abseiling 6
Targets:  abseiling 7
Targets:  abseiling 8
Targets:  abseiling 9
Targets:  abseiling 10
Targets:  abseiling 11
Targets:  abseiling 12
Targets:  abseiling 13
Targets:  abseiling 14
Targets:  abseiling 15
Targets:  abseiling 16
Targets:  abseiling 17
Targets:  abseiling 18
Targets:  abseiling 19
Targets:  abseiling 20
Targets:  abseiling 21
Targets:  abseiling 22
Targets:  abseiling 23
Targets:  abseiling 24
Targets:  abseiling 25
Targets:  abseiling 26
Targets:  abseiling 27
Targets:  abseiling 28
Targets:  abseiling 29
Targets:  abseiling 30
Targets:  abseiling 31
Targets:  abseiling 32
Targets:  abseiling 33
Targets:  abseiling 34
Targets:  abseiling 35

Class_folder:  air drumming 

Targets:  air drumming 0
Targets:  air drumming 1
Targets:  air drumming 2
Targets:  air drumming 3
Targets:  air drumming 

In [2]:
Data_Loader

{'train': <torch.utils.data.dataloader.DataLoader at 0x2d5964ad8b0>,
 'val': <torch.utils.data.dataloader.DataLoader at 0x2d5ced58fd0>}

In [3]:
# # Print the Data_Loader
# for phase, dataloader in Data_Loader.items():
#     #print(f"{phase.capitalize()} Dataloader:")
#     for inputs, labels in dataloader:
#         print(f"Batch size: {inputs.size(0)}")
#         print(f"Number of batches: {len(dataloader)}")
#         break  # Print the first batch only for brevity
#     print("\n")
#     if phase == 5:
#         break

In [16]:
class VisionTransformer(nn.Module):
    def __init__(self, num_classes, NumFrames, SequenceLength):
        super(VisionTransformer, self).__init__()
        self.NumFrames = NumFrames
        self.SequenceLength = SequenceLength
        self.NumofPatches = NumFrames * SequenceLength
        self.patch_dim = input_size // 16  # Assuming patch size of 16x16

        # Define embed_dim and num_heads based on your data and task
        self.embed_dim = 224  # Adjust based on your data
        self.num_heads = 8  # Adjust based on your data

        if self.embed_dim % self.num_heads != 0:
            raise AssertionError("embedded dim must be divisible by num of heads")

        # Positional embeddings
        self.positional_embedding = nn.Parameter(torch.randn(1, self.NumofPatches, self.embed_dim))

        # Transformer encoder layers
        self.transformer_layers = nn.ModuleList([
            nn.TransformerEncoderLayer(d_model=self.embed_dim, nhead=self.num_heads)
            for _ in range(2)  # Use 2 layers as an example; you can adjust as needed ])

        # Classification head
        self.classification_head = nn.Linear(self.NumFrames * self.embed_dim, num_classes)
    def forward(self, x): # Concatenate the list of frames along the first dimension
                                   
        x = torch.cat(x, dim=0)
        
        expected_size = (x.size(0), self.NumofPatches, self.embed_dim) # Calculate the expected size of positional embeddings
        
        positional_embeddings = self.positional_embedding.expand(expected_size) # Resize the positional embeddings to match the expected size
       
        x = x + positional_embeddings
        x = x.view(self.NumFrames, self.SequenceLength, -1).permute(1, 0, 2)          # Reshape for input to the transformer

        # Pass through the transformer layers
        for layer in self.transformer_layers:
            x = layer(x)
    
        x = x.permute(1, 0, 2).reshape(-1, self.NumFrames * self.embed_dim)  # Reshape and pass through the classification head
        x = self.classification_head(x)
    
        return x


In [18]:
# calling the ViT model
model = VisionTransformer(num_classes, num_frames, sequence_length)

criterion = nn.CrossEntropyLoss()                             # Loss function and optimizer
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training the model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [19]:
for epoch in range(epochs_count):
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in Data_Loader[phase]:
            inputs = [frame.to(device) for frame in inputs]
        
            # Create a LabelEncoder instance
            LabelEncoder = LabelEncoder()
            
            # Fit and transform the labels to obtain encoded values
            labels = LabelEncoder.fit_transform(labels)
            # print("type:", type(encoded_labels))
            # print("Labels",  labels)
            # print(len(labels))

            
            labels = torch.tensor(labels).to(device)  # Convert labels to a tensor and move to the device
            print(type(labels))
        
        
            optimizer.zero_grad()
        
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
        
                if phase == 'train':
                    loss.backward()
                    optimizer.step()
        
            running_loss += loss.item() * inputs[0].size(0)
            running_corrects += torch.sum(preds == labels.data)
            # Empty the array for the next iteration
            # my_array = np.labels([])
        EpochLoss = running_loss / len(image_datasets[phase])
        EpochAccuracy = running_corrects.double() / len(image_datasets[phase])


        print(f'{phase} Loss: {EpochLoss:.4f} Acc: {EpochAccuracy:.4f}')

# Saving the model
torch.save(model.state_dict(), 'video_classifier_model.pth')

<class 'torch.Tensor'>


RuntimeError: The size of tensor a (224) must match the size of tensor b (75) at non-singleton dimension 2

In [ ]:
from sklearn.preprocessing import LabelEncoder